In [1]:
from PIL import Image
import numpy as np

In [ ]:
from google.colab import files
files.upload()

In [3]:
import zipfile
path = './yalefaces.zip'
zip_object = zipfile.ZipFile(file=path, mode='r')
zip_object.extractall('./')
zip_object.close()

In [ ]:
import os
os.listdir('./yalefaces/train')

In [5]:
def image_data():
  paths = [os.path.join('./yalefaces/train', f) for f in os.listdir('./yalefaces/train')]
  faces = []
  ids = []
  for path in paths:
    if path == './yalefaces/train/.ipynb_checkpoints':
      continue
    image = Image.open(path).convert('L') # L = black and white image
    image_np = np.array(image, 'uint8')
    id = int(os.path.split(path)[1].split('.')[0].replace('subject', ''))
    ids.append(id)
    faces.append(image_np)
  return np.array(ids), faces

In [6]:
ids, faces = image_data()

In [7]:
np.unique(ids)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15])

In [8]:
import tensorflow as tf
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(faces, ids, test_size=0.2, random_state=3)
X_train, X_test, y_train, y_test = np.array(X_train), np.array(X_test), np.array(y_train), np.array(y_test)

In [9]:
X_train.shape, X_test.shape

((108, 243, 320), (27, 243, 320))

In [ ]:
y_train.shape, y_test.shape

In [11]:
from keras.utils import np_utils

X_train = X_train.reshape(X_train.shape[0], 243, 320, 1)
X_test = X_test.reshape(X_test.shape[0], 243, 320, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
n_classes = len(np.unique(y_train)) + 1
y_train = np_utils.to_categorical(y_train, n_classes)
y_test = np_utils.to_categorical(y_test, n_classes)

In [12]:
X_train.shape, X_test.shape

((108, 243, 320, 1), (27, 243, 320, 1))

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D
from keras.layers import MaxPool2D
from keras.layers import Flatten
from keras.layers import Dense
 
classifier = Sequential()
classifier.add(Convolution2D(32, kernel_size=(5, 5), strides=(1, 1), input_shape=(243, 320, 1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Convolution2D(64, kernel_size=(5, 5), strides=(1, 1), activation='relu'))
classifier.add(MaxPool2D(pool_size=(2,2)))
classifier.add(Flatten())
classifier.add(Dense(64, activation='relu'))
classifier.add(Dense(16, activation='softmax'))
classifier.compile(loss='categorical_crossentropy', optimizer = 'adam', metrics=["accuracy"])
classifier.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 239, 316, 32)      832       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 119, 158, 32)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 115, 154, 64)      51264     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 57, 77, 64)       0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 280896)            0         
                                                                 
 dense (Dense)               (None, 64)                1

In [14]:
classifier.fit(X_train, y_train, epochs=10, batch_size=10)

Epoch 1/10
11/11 [==============================] - 25s 2s/step - loss: 549.3214 - accuracy: 0.2222
Epoch 2/10
11/11 [==============================] - 20s 2s/step - loss: 0.6027 - accuracy: 0.8704
Epoch 3/10
11/11 [==============================] - 20s 2s/step - loss: 0.0431 - accuracy: 0.9907
Epoch 4/10
11/11 [==============================] - 20s 2s/step - loss: 0.0551 - accuracy: 0.9907
Epoch 5/10
11/11 [==============================] - 21s 2s/step - loss: 1.7160 - accuracy: 0.8981
Epoch 6/10
11/11 [==============================] - 20s 2s/step - loss: 0.4988 - accuracy: 0.9074
Epoch 7/10
11/11 [==============================] - 20s 2s/step - loss: 0.1263 - accuracy: 0.9815
Epoch 8/10
11/11 [==============================] - 20s 2s/step - loss: 0.0231 - accuracy: 1.0000
Epoch 9/10
11/11 [==============================] - 20s 2s/step - loss: 0.0124 - accuracy: 1.0000
Epoch 10/10
11/11 [==============================] - 20s 2s/step - loss: 0.0057 - accuracy: 1.0000


In [43]:
predictions = classifier.predict(X_test)

In [44]:
def transform_prediction(prediction):
  prediction = list(prediction)
  max_value = max(prediction)
  return prediction.index(max_value)

In [45]:
predictions = list(map(lambda x: transform_prediction(x), predictions))
y_test = list(map(lambda x: transform_prediction(x), y_test))

In [52]:
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions))

              precision    recall  f1-score   support

           1       1.00      0.75      0.86         4
           2       0.50      0.50      0.50         2
           3       1.00      0.33      0.50         3
           4       0.67      1.00      0.80         2
           5       1.00      1.00      1.00         1
           7       0.50      1.00      0.67         2
           8       0.50      1.00      0.67         1
           9       1.00      0.33      0.50         3
          10       0.50      1.00      0.67         1
          11       1.00      1.00      1.00         1
          12       1.00      0.50      0.67         2
          13       1.00      1.00      1.00         2
          14       0.50      1.00      0.67         1
          15       1.00      1.00      1.00         2

    accuracy                           0.74        27
   macro avg       0.80      0.82      0.75        27
weighted avg       0.85      0.74      0.73        27

